In [1]:
import pandas as pd

# Load the datasets
providers = pd.read_csv('providers_data.csv')
receivers = pd.read_csv('receivers_data.csv')
food_listings = pd.read_csv('food_listings_data.csv')
claims = pd.read_csv('claims_data.csv')

# Check data samples and info
print(providers.head())
print(providers.info())

# Convert date columns to datetime format
food_listings['Expiry_Date'] = pd.to_datetime(food_listings['Expiry_Date'])
claims['Timestamp'] = pd.to_datetime(claims['Timestamp'])

   Provider_ID                         Name           Type  \
0            1             Gonzales-Cochran    Supermarket   
1            2  Nielsen, Johnson and Fuller  Grocery Store   
2            3                 Miller-Black    Supermarket   
3            4   Clark, Prince and Williams  Grocery Store   
4            5               Coleman-Farley  Grocery Store   

                                             Address            City  \
0  74347 Christopher Extensions\nAndreamouth, OK ...     New Jessica   
1           91228 Hanson Stream\nWelchtown, OR 27136     East Sheena   
2  561 Martinez Point Suite 507\nGuzmanchester, W...  Lake Jesusview   
3     467 Bell Trail Suite 409\nPort Jesus, IA 61188     Mendezmouth   
4  078 Matthew Creek Apt. 319\nSaraborough, MA 53978   Valentineside   

                Contact  
0       +1-600-220-0480  
1  +1-925-283-8901x6297  
2      001-517-295-2206  
3      556.944.8935x401  
4          193.714.6577  
<class 'pandas.core.frame.DataFrame'>


In [2]:
import sqlite3

conn = sqlite3.connect('food_wastage.db')
cursor = conn.cursor()

# Create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS Providers (
    Provider_ID INTEGER PRIMARY KEY,
    Name TEXT,
    Type TEXT,
    Address TEXT,
    City TEXT,
    Contact TEXT
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Receivers (
    Receiver_ID INTEGER PRIMARY KEY,
    Name TEXT,
    Type TEXT,
    City TEXT,
    Contact TEXT
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Food_Listings (
    Food_ID INTEGER PRIMARY KEY,
    Food_Name TEXT,
    Quantity INTEGER,
    Expiry_Date DATE,
    Provider_ID INTEGER,
    Provider_Type TEXT,
    Location TEXT,
    Food_Type TEXT,
    Meal_Type TEXT,
    FOREIGN KEY (Provider_ID) REFERENCES Providers (Provider_ID)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Claims (
    Claim_ID INTEGER PRIMARY KEY,
    Food_ID INTEGER,
    Receiver_ID INTEGER,
    Status TEXT,
    Timestamp DATETIME,
    FOREIGN KEY (Food_ID) REFERENCES Food_Listings (Food_ID),
    FOREIGN KEY (Receiver_ID) REFERENCES Receivers (Receiver_ID)
);
''')

conn.commit()

In [3]:
providers.to_sql('Providers', conn, if_exists='replace', index=False)
receivers.to_sql('Receivers', conn, if_exists='replace', index=False)
food_listings.to_sql('Food_Listings', conn, if_exists='replace', index=False)
claims.to_sql('Claims', conn, if_exists='replace', index=False)

1000

In [5]:
import sqlite3
import pandas as pd

def run_food_wastage_queries(db_path='food_wastage.db', output_file='query_results.txt'):
    conn = sqlite3.connect(db_path)

    queries = {
        # Food Providers & Receivers
        "1. Providers per city": """
            SELECT City, COUNT(DISTINCT Provider_ID) AS Num_Providers FROM Providers GROUP BY City;
        """,
        "1. Receivers per city": """
            SELECT City, COUNT(DISTINCT Receiver_ID) AS Num_Receivers FROM Receivers GROUP BY City;
        """,
        "2. Top provider types by food contribution": """
            SELECT P.Type, COUNT(F.Food_ID) AS Food_Contribution
            FROM Providers P
            JOIN Food_Listings F ON P.Provider_ID = F.Provider_ID
            GROUP BY P.Type
            ORDER BY Food_Contribution DESC;
        """,
        "3. Provider contacts in a city (example: 'CityX')": """
            SELECT Name, Contact FROM Providers WHERE City = 'CityX';
        """,
        "4. Receivers with most claims": """
            SELECT R.Name, COUNT(C.Claim_ID) AS Claimed_Foods
            FROM Receivers R
            JOIN Claims C ON R.Receiver_ID = C.Receiver_ID
            GROUP BY R.Receiver_ID
            ORDER BY Claimed_Foods DESC;
        """,

        # Food Listings & Availability
        "5. Total food quantity available": """
            SELECT SUM(Quantity) FROM Food_Listings;
        """,
        "6. City with highest food listings": """
            SELECT Location AS City, COUNT(Food_ID) AS Num_Food_Listings
            FROM Food_Listings
            GROUP BY Location
            ORDER BY Num_Food_Listings DESC;
        """,
        "7. Most common food types": """
            SELECT Food_Type, COUNT(Food_ID) AS Count
            FROM Food_Listings
            GROUP BY Food_Type
            ORDER BY Count DESC;
        """,

        # Claims & Distribution
        "8. Number of claims per food item": """
            SELECT F.Food_Name, COUNT(C.Claim_ID) AS Num_Claims
            FROM Food_Listings F
            LEFT JOIN Claims C ON F.Food_ID = C.Food_ID
            GROUP BY F.Food_Name
            ORDER BY Num_Claims DESC;
        """,
        "9. Provider with highest successful claims": """
            SELECT P.Name, COUNT(C.Claim_ID) AS Completed_Claims
            FROM Providers P
            JOIN Food_Listings F ON P.Provider_ID = F.Provider_ID
            JOIN Claims C ON F.Food_ID = C.Food_ID
            WHERE C.Status = 'Completed'
            GROUP BY P.Provider_ID
            ORDER BY Completed_Claims DESC;
        """,
        "10. Claim status percentages": """
            SELECT Status, 
                   ROUND( (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM Claims)), 2 ) AS Percentage
            FROM Claims
            GROUP BY Status;
        """,

        # Analysis & Insights
        "11. Average quantity claimed per receiver": """
            SELECT R.Name, AVG(F.Quantity) AS Avg_Quantity_Claimed
            FROM Receivers R
            JOIN Claims C ON R.Receiver_ID = C.Receiver_ID
            JOIN Food_Listings F ON C.Food_ID = F.Food_ID
            GROUP BY R.Receiver_ID
            ORDER BY Avg_Quantity_Claimed DESC;
        """,
        "12. Most claimed meal type": """
            SELECT F.Meal_Type, COUNT(C.Claim_ID) AS Count
            FROM Food_Listings F
            JOIN Claims C ON F.Food_ID = C.Food_ID
            GROUP BY F.Meal_Type
            ORDER BY Count DESC;
        """,
        "13. Total quantity donated per provider": """
            SELECT P.Name, SUM(F.Quantity) AS Total_Quantity
            FROM Providers P
            JOIN Food_Listings F ON P.Provider_ID = F.Provider_ID
            GROUP BY P.Provider_ID
            ORDER BY Total_Quantity DESC;
        """
    }

    with open(output_file, 'w', encoding='utf-8') as f:
        for title, query in queries.items():
            f.write("="*80 + "\n")
            f.write(f"Query: {title}\n\n")
            try:
                df = pd.read_sql_query(query, conn)
                if df.empty:
                    f.write("No results returned.\n\n")
                else:
                    f.write(df.to_string(index=False) + "\n\n")
            except Exception as e:
                f.write(f"Error executing query:\n{str(e)}\n\n")

    conn.close()
    print(f"All query results have been successfully written to '{output_file}'.")

if __name__ == "__main__":
    run_food_wastage_queries()


All query results have been successfully written to 'query_results.txt'.
